In [1]:
import functools
import glob
import matplotlib.colors as mc
import nltk
import numpy as np
import operator
import os
import pandas as pd
import re
import string
import textstat
import unicodedata

from collections import Counter
from lexical_diversity import lex_div as ld
from lexicalrichness import LexicalRichness
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tag import pos_tag, map_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.cluster import DBSCAN
from sklearn import metrics
from xml.dom import minidom
from xml.etree import ElementTree as ET
from lxml import etree

In [2]:
df = pd.read_csv("analisis_exploratorio.csv")

In [12]:
df

,index,suspicious,fleshReadingEase,numMedWord,syllableCount,numLetters,numOfPunctN,typeToken,funcWords
0,1,suspicious-document00001.txt,-0.312325,0.015679,0.014479,0.018852,0.009581,0.260457,0.382564
1,2,suspicious-document00002.txt,0.503989,0.018039,0.015125,0.020672,0.011742,0.151873,0.381168
2,3,suspicious-document00003.txt,-0.695951,0.023506,0.021008,0.026764,0.013321,0.176936,0.371205
3,4,suspicious-document00004.txt,0.707718,0.009878,0.007149,0.010041,0.007700,0.362661,0.354160
4,5,suspicious-document00005.txt,-1.189170,0.009845,0.009216,0.012083,0.006138,0.263451,0.323767
...,...,...,...,...,...,...,...,...,...
4748,4749,suspicious-document04749.txt,0.562625,0.022726,0.018606,0.025787,0.014360,0.235541,0.374746
4749,4750,suspicious-document04750.txt,0.821500,0.007667,0.004789,0.006691,0.008501,0.277382,0.346781
4750,4751,suspicious-document04751.txt,0.359892,0.037887,0.034702,0.045580,0.018778,0.082705,0.395972
4751,4752,suspicious-document04752.txt,-0.876147,0.011374,0.011891,0.013248,0.007525,0.342222,0.362937


In [3]:
 df.describe()

,index,fleshReadingEase,numMedWord,syllableCount,numLetters,numOfPunctN,typeToken,funcWords
count,4753.000000,4753.000000,4753.000000,4753.000000,4753.000000,4753.000000,4753.000000,4753.000000
mean,2377.000000,24.686104,103.922102,122.864963,398.216866,15.724600,0.241779,0.362971
std,1372.217245,96.600555,113.491020,158.739552,426.901425,21.405299,0.127579,0.026147
min,1.000000,-2928.530000,5.450980,5.423358,19.490196,2.000000,0.027882,0.087025
25%,1189.000000,20.120000,58.986842,64.493663,213.035714,10.125091,0.126569,0.350163
50%,2377.000000,54.360000,79.108434,90.930886,295.414634,12.719101,0.226050,0.364212
75%,3565.000000,67.220000,122.066770,147.166667,479.894578,17.476190,0.333575,0.378753
max,4753.000000,100.280000,6018.000000,8817.000000,21279.000000,1308.000000,0.736842,0.459964


In [4]:
# Feature Selection
from sklearn.feature_selection import VarianceThreshold

In [5]:
df["fleshReadingEase"] = df["fleshReadingEase"] / df["fleshReadingEase"].max()
df["numMedWord"] = df["numMedWord"] / df["numMedWord"].max()
df["syllableCount"] = df["syllableCount"] / df["syllableCount"].max()
df["numLetters"] = df["numLetters"] / df["numLetters"].max()
df["numOfPunctN"] = df["numOfPunctN"] / df["numOfPunctN"].max()

In [6]:
 df2 = df.iloc[:,[2,3,4,5,6,7,8]]

In [13]:
df

,index,suspicious,fleshReadingEase,numMedWord,syllableCount,numLetters,numOfPunctN,typeToken,funcWords
0,1,suspicious-document00001.txt,-0.312325,0.015679,0.014479,0.018852,0.009581,0.260457,0.382564
1,2,suspicious-document00002.txt,0.503989,0.018039,0.015125,0.020672,0.011742,0.151873,0.381168
2,3,suspicious-document00003.txt,-0.695951,0.023506,0.021008,0.026764,0.013321,0.176936,0.371205
3,4,suspicious-document00004.txt,0.707718,0.009878,0.007149,0.010041,0.007700,0.362661,0.354160
4,5,suspicious-document00005.txt,-1.189170,0.009845,0.009216,0.012083,0.006138,0.263451,0.323767
...,...,...,...,...,...,...,...,...,...
4748,4749,suspicious-document04749.txt,0.562625,0.022726,0.018606,0.025787,0.014360,0.235541,0.374746
4749,4750,suspicious-document04750.txt,0.821500,0.007667,0.004789,0.006691,0.008501,0.277382,0.346781
4750,4751,suspicious-document04751.txt,0.359892,0.037887,0.034702,0.045580,0.018778,0.082705,0.395972
4751,4752,suspicious-document04752.txt,-0.876147,0.011374,0.011891,0.013248,0.007525,0.342222,0.362937


In [14]:
thresholder = VarianceThreshold(threshold=.001)

In [15]:
X_high_variance = thresholder.fit_transform(df2)

In [16]:
X_high_variance

array([[-0.31232549,  0.26045701],
       [ 0.50398883,  0.15187297],
       [-0.69595134,  0.17693596],
       ...,
       [ 0.3598923 ,  0.08270455],
       [-0.87614679,  0.34222222],
       [-2.05624252,  0.22632862]])